# Merge Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from datetime import datetime
%matplotlib inline

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

# Display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

## load and merge stocks

In [7]:
# Define the data path
data_path = "/Users/daniellott2/Library/CloudStorage/GoogleDrive-dlott@arizona.edu/My Drive/DATA 462/Final Project/data/"

# Find all CSV files with the pattern "HistoricalData_*.csv"
csv_files = glob.glob(os.path.join(data_path, "HistoricalData_*.csv"))
print(f"Found {len(csv_files)} CSV files to process")

# Function to load and process a single CSV file
def process_csv(file_path):
    # Extract ticker symbol from filename
    ticker = os.path.basename(file_path).split('_')[1].split('.')[0] if '_' in os.path.basename(file_path) else f"Stock_{os.path.basename(file_path)[:8]}"
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Only keep Date and Open columns
    if 'Open' in df.columns:
        return df[['Date', 'Open']].rename(columns={'Open': ticker})
    else:
        print(f"Warning: No 'Open' column found in {file_path}")
        return None

# Process all CSV files and store DataFrames in a list
dfs = []
for file in csv_files:
    try:
        print(f"Processing {os.path.basename(file)}...")
        df = process_csv(file)
        if df is not None:
            # Ensure numeric formatting for open price
            df[df.columns[1]] = pd.to_numeric(df[df.columns[1]].astype(str).str.replace(',', ''), errors='coerce')
            dfs.append(df)
            print(f"  - Added {df.shape[0]} rows for ticker: {df.columns[1]}")
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")

# Merge all DataFrames on the Date column
if dfs:
    # Start with the first dataframe
    merged_df = dfs[0]
    
    # Merge with the rest one by one using left outer join
    for df in dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='Date', how='outer')
    
    print(f"\nFinal merged DataFrame shape: {merged_df.shape}")
    
    # Sort by date
    merged_df = merged_df.sort_values('Date')
    
    # Save the merged DataFrame
    pivot_csv_path = os.path.join(data_path, "combined_stocks.csv")
    merged_df.to_csv(pivot_csv_path, index=False)
    print(f"Pivot table saved to: {pivot_csv_path}")
    
    # Display some basic information
    print(f"\nDate range: {merged_df['Date'].min()} to {merged_df['Date'].max()}")
    print(f"Number of dates: {merged_df['Date'].nunique()}")
    print(f"Number of tickers: {len(merged_df.columns) - 1}")  # Subtract 1 for the Date column
    print(f"Tickers: {', '.join(merged_df.columns[1:])}")
else:
    print("No dataframes were successfully processed.")

Found 10 CSV files to process
Processing HistoricalData_XLB.csv...
  - Added 2516 rows for ticker: XLB
Processing HistoricalData_XLU.csv...
  - Added 2516 rows for ticker: XLU
Processing HistoricalData_XLV.csv...
  - Added 2516 rows for ticker: XLV
Processing HistoricalData_XLE.csv...
  - Added 2516 rows for ticker: XLE
Processing HistoricalData_XLF.csv...
  - Added 2516 rows for ticker: XLF
Processing HistoricalData_XLP.csv...
  - Added 2516 rows for ticker: XLP
Processing HistoricalData_XLRE.csv...
  - Added 2390 rows for ticker: XLRE
Processing HistoricalData_XLK.csv...
  - Added 2516 rows for ticker: XLK
Processing HistoricalData_XLI.csv...
  - Added 2516 rows for ticker: XLI
Processing HistoricalData_XLY.csv...
  - Added 2516 rows for ticker: XLY

Final merged DataFrame shape: (2516, 11)
Pivot table saved to: /Users/daniellott2/Library/CloudStorage/GoogleDrive-dlott@arizona.edu/My Drive/DATA 462/Final Project/data/combined_stocks.csv

Date range: 2015-04-10 00:00:00 to 2025-04-09 

## add ffr data

In [9]:
def process_ffr_data(ffr_file_path):
    """
    Load monthly Federal Funds Rate data and convert it to daily frequency
    by forward-filling missing dates.
    """
    # Load the FFR data
    ffr_df = pd.read_csv(ffr_file_path)
    
    # Convert date column to datetime (assuming it has a date column)
    date_col = [col for col in ffr_df.columns if 'date' in col.lower() or 'time' in col.lower()]
    if date_col:
        date_col = date_col[0]
    else:
        date_col = ffr_df.columns[0]  # Assume first column is date
    
    # Rename the date column to 'Date' for consistency
    ffr_df = ffr_df.rename(columns={date_col: 'Date'})
    
    # Convert to datetime
    ffr_df['Date'] = pd.to_datetime(ffr_df['Date'])
    
    # Rename the FFR column to 'FFR' if not already named that
    rate_col = [col for col in ffr_df.columns if col != 'Date'][0]  # Assume the non-date column is the rate
    ffr_df = ffr_df.rename(columns={rate_col: 'FFR'})
    
    # Select only Date and FFR columns
    ffr_df = ffr_df[['Date', 'FFR']]
    
    # Create a date range spanning the entire period of stock data
    date_range = pd.date_range(
        start=merged_df['Date'].min(), 
        end=merged_df['Date'].max(), 
        freq='D'
    )
    
    # Create a DataFrame with all dates
    full_date_df = pd.DataFrame({'Date': date_range})
    
    # Merge with the FFR data
    daily_ffr = pd.merge(full_date_df, ffr_df, on='Date', how='left')
    
    # Forward fill missing values (use the last known FFR rate)
    daily_ffr['FFR'] = daily_ffr['FFR'].ffill()
    
    return daily_ffr

# Path to FFR data file - update this to your actual file path
ffr_file_path = os.path.join(data_path, "FFR.csv")

try:
    print(f"Processing Federal Funds Rate data...")
    daily_ffr_df = process_ffr_data(ffr_file_path)
    print(f"Expanded FFR data from monthly to daily frequency")
    print(f"FFR date range: {daily_ffr_df['Date'].min()} to {daily_ffr_df['Date'].max()}")
    
    # Merge FFR data with stock data
    merged_df = pd.merge(merged_df, daily_ffr_df, on='Date', how='left')
    
    # Update the saved file with FFR data included
    merged_csv_path = os.path.join(data_path, "combined_stocks_with_ffr.csv")
    merged_df.to_csv(merged_csv_path, index=False)
    print(f"Combined data with FFR saved to: {merged_csv_path}")
    
    # Display some basic information about the FFR data
    print(f"\nFFR data summary:")
    print(f"Min: {merged_df['FFR'].min():.2f}%, Max: {merged_df['FFR'].max():.2f}%")
    print(f"Mean: {merged_df['FFR'].mean():.2f}%, Median: {merged_df['FFR'].median():.2f}%")
    
except Exception as e:
    print(f"Error processing FFR data: {str(e)}")

Processing Federal Funds Rate data...
Expanded FFR data from monthly to daily frequency
FFR date range: 2015-04-10 00:00:00 to 2025-04-09 00:00:00
Combined data with FFR saved to: /Users/daniellott2/Library/CloudStorage/GoogleDrive-dlott@arizona.edu/My Drive/DATA 462/Final Project/data/combined_stocks_with_ffr.csv

FFR data summary:
Min: 0.05%, Max: 5.33%
Mean: 1.94%, Median: 1.30%


## add mortgage rates

In [10]:
def process_mortgage_rates(mortgage_file_path):
    """
    Load weekly mortgage rate data and convert it to daily frequency
    by forward-filling missing dates.
    """
    print(f"Processing mortgage rate data...")
    
    # Load the mortgage rate data
    mortgage_df = pd.read_csv(mortgage_file_path)
    
    # Convert date column to datetime
    mortgage_df['observation_date'] = pd.to_datetime(mortgage_df['observation_date'])
    
    # Rename columns for consistency
    mortgage_df = mortgage_df.rename(columns={'observation_date': 'Date', 'MORTGAGE30US': 'Mortgage30Y'})
    
    # Create a date range spanning the entire period of stock data
    date_range = pd.date_range(
        start=merged_df['Date'].min(),
        end=merged_df['Date'].max(),
        freq='D'
    )
    
    # Create a DataFrame with all dates
    full_date_df = pd.DataFrame({'Date': date_range})
    
    # Merge with the mortgage data
    daily_mortgage = pd.merge(full_date_df, mortgage_df, on='Date', how='left')
    
    # Forward fill missing values (use the last known mortgage rate)
    daily_mortgage['Mortgage30Y'] = daily_mortgage['Mortgage30Y'].ffill()
    
    print(f"Expanded mortgage rate data from weekly to daily frequency")
    print(f"Mortgage date range: {daily_mortgage['Date'].min()} to {daily_mortgage['Date'].max()}")
    
    return daily_mortgage

# Path to mortgage rate data file
mortgage_file_path = os.path.join(data_path, "MORTGAGE30US.csv")

try:
    # Process mortgage rates data
    daily_mortgage_df = process_mortgage_rates(mortgage_file_path)
    
    # Merge mortgage data with the existing dataset
    merged_df = pd.merge(merged_df, daily_mortgage_df, on='Date', how='left')
    
    # Display some basic information about the mortgage data
    print(f"\nMortgage rate data summary:")
    print(f"Min: {merged_df['Mortgage30Y'].min():.2f}%, Max: {merged_df['Mortgage30Y'].max():.2f}%")
    print(f"Mean: {merged_df['Mortgage30Y'].mean():.2f}%, Median: {merged_df['Mortgage30Y'].median():.2f}%")
    
    # Update the saved file with mortgage data included
    final_csv_path = os.path.join(data_path, "combined_stocks_with_ffr_mortgage.csv")
    merged_df.to_csv(final_csv_path, index=False)
    print(f"Combined data with FFR and mortgage rates saved to: {final_csv_path}")
    
except Exception as e:
    print(f"Error processing mortgage rate data: {str(e)}")

Processing mortgage rate data...
Expanded mortgage rate data from weekly to daily frequency
Mortgage date range: 2015-04-10 00:00:00 to 2025-04-09 00:00:00

Mortgage rate data summary:
Min: 2.65%, Max: 7.79%
Mean: 4.57%, Median: 4.02%
Combined data with FFR and mortgage rates saved to: /Users/daniellott2/Library/CloudStorage/GoogleDrive-dlott@arizona.edu/My Drive/DATA 462/Final Project/data/combined_stocks_with_ffr_mortgage.csv


In [ ]:
## add ZHVI data (home price)

In [11]:
def process_zhvi_data(zhvi_file_path, states_to_include=['Florida', 'Arizona']):
    """
    Load monthly ZHVI (Zillow Home Value Index) data for specific states and 
    convert it to daily frequency by forward-filling missing dates.
    """
    print(f"Processing ZHVI housing price data...")
    
    # Load the ZHVI data
    zhvi_df = pd.read_csv(zhvi_file_path)
    
    # Filter for specified states only
    zhvi_df = zhvi_df[zhvi_df['RegionName'].isin(states_to_include)]
    
    if len(zhvi_df) == 0:
        raise ValueError(f"No data found for states: {states_to_include}")
        
    # Melt the dataframe to convert from wide to long format
    # First, identify date columns (they should be in YYYY-MM-DD format)
    date_columns = [col for col in zhvi_df.columns if '-' in col]
    
    # Melt the dataframe
    zhvi_melted = pd.melt(
        zhvi_df,
        id_vars=['RegionName'],
        value_vars=date_columns,
        var_name='Date',
        value_name='ZHVI'
    )
    
    # Convert to datetime
    zhvi_melted['Date'] = pd.to_datetime(zhvi_melted['Date'])
    
    # Create a dataframe for each state
    state_dfs = {}
    for state in states_to_include:
        state_data = zhvi_melted[zhvi_melted['RegionName'] == state].copy()
        
        # Create column name with state prefix
        column_name = f"{state}_ZHVI"
        
        # Create a date range spanning the entire period of stock data
        date_range = pd.date_range(
            start=merged_df['Date'].min(),
            end=merged_df['Date'].max(),
            freq='D'
        )
        
        # Create a DataFrame with all dates
        full_date_df = pd.DataFrame({'Date': date_range})
        
        # Merge with the ZHVI data
        state_df = pd.merge(full_date_df, 
                           state_data[['Date', 'ZHVI']], 
                           on='Date', 
                           how='left')
        
        # Forward fill missing values
        state_df['ZHVI'] = state_df['ZHVI'].ffill()
        
        # Rename for clarity
        state_df = state_df.rename(columns={'ZHVI': column_name})
        
        state_dfs[state] = state_df
    
    # Merge all state dataframes together
    result_df = state_dfs[states_to_include[0]]
    for state in states_to_include[1:]:
        result_df = pd.merge(result_df, state_dfs[state], on='Date', how='outer')
    
    print(f"Expanded ZHVI data from monthly to daily frequency")
    print(f"ZHVI date range: {result_df['Date'].min()} to {result_df['Date'].max()}")
    
    return result_df

# Path to ZHVI data file
zhvi_file_path = os.path.join(data_path, "ZHVI.csv")

try:
    # Process ZHVI data
    daily_zhvi_df = process_zhvi_data(zhvi_file_path, states_to_include=['Florida', 'Arizona'])
    
    # Merge ZHVI data with the existing dataset
    merged_df = pd.merge(merged_df, daily_zhvi_df, on='Date', how='left')
    
    # Display some basic information about the ZHVI data
    print(f"\nFlorida ZHVI data summary:")
    print(f"Min: ${merged_df['Florida_ZHVI'].min():.2f}, Max: ${merged_df['Florida_ZHVI'].max():.2f}")
    print(f"Mean: ${merged_df['Florida_ZHVI'].mean():.2f}, Median: ${merged_df['Florida_ZHVI'].median():.2f}")
    
    print(f"\nArizona ZHVI data summary:")
    print(f"Min: ${merged_df['Arizona_ZHVI'].min():.2f}, Max: ${merged_df['Arizona_ZHVI'].max():.2f}")
    print(f"Mean: ${merged_df['Arizona_ZHVI'].mean():.2f}, Median: ${merged_df['Arizona_ZHVI'].median():.2f}")
    
    # Update the saved file with ZHVI data included
    final_csv_path = os.path.join(data_path, "combined_stocks_with_ffr_mortgage_zhvi.csv")
    merged_df.to_csv(final_csv_path, index=False)
    print(f"Combined data with FFR, mortgage rates, and ZHVI saved to: {final_csv_path}")
    
except Exception as e:
    print(f"Error processing ZHVI data: {str(e)}")
    import traceback
    traceback.print_exc()

Processing ZHVI housing price data...
Expanded ZHVI data from monthly to daily frequency
ZHVI date range: 2015-04-10 00:00:00 to 2025-04-09 00:00:00

Florida ZHVI data summary:
Min: $175102.09, Max: $394769.17
Mean: $279632.43, Median: $249868.93

Arizona ZHVI data summary:
Min: $196768.81, Max: $456005.45
Mean: $315963.88, Median: $283642.18
Combined data with FFR, mortgage rates, and ZHVI saved to: /Users/daniellott2/Library/CloudStorage/GoogleDrive-dlott@arizona.edu/My Drive/DATA 462/Final Project/data/combined_stocks_with_ffr_mortgage_zhvi.csv
